In [2]:
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
scipy: 1.0.0
numpy: 1.14.0
matplotlib: 2.1.2
pandas: 0.22.0
sklearn: 0.19.1


In [3]:
import pandas as pd 
import numpy as np
from pandas import DataFrame as df
import matplotlib
# from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import folium
from ipykernel import kernelapp as app
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm

In [4]:
data = pd.read_csv("WineReview.csv", index_col=0)
print(data.describe().transpose())
print(data.info())

           count       mean        std   min   25%   50%   75%     max
points  129971.0  88.447138   3.039730  80.0  86.0  88.0  91.0   100.0
price   120975.0  35.363389  41.022218   4.0  17.0  25.0  42.0  3300.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 

In [5]:
data1 = data[['title','country','province','taster_name','variety','price','points','description','designation','region_1','region_2','taster_twitter_handle','winery']]
data1.head(5)

,title,country,province,taster_name,variety,price,points,description,designation,region_1,region_2,taster_twitter_handle,winery
0,Nicosia 2013 Vulkà Bianco (Etna),Italy,Sicily & Sardinia,Kerin O’Keefe,White Blend,NaN,87,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,Etna,NaN,@kerinokeefe,Nicosia
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portugal,Douro,Roger Voss,Portuguese Red,15.0,87,"This is ripe and fruity, a wine that is smooth...",Avidagos,NaN,NaN,@vossroger,Quinta dos Avidagos
2,Rainstorm 2013 Pinot Gris (Willamette Valley),US,Oregon,Paul Gregutt,Pinot Gris,14.0,87,"Tart and snappy, the flavors of lime flesh and...",NaN,Willamette Valley,Willamette Valley,@paulgwine,Rainstorm
3,St. Julian 2013 Reserve Late Harvest Riesling ...,US,Michigan,Alexander Peartree,Riesling,13.0,87,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,Lake Michigan Shore,NaN,NaN,St. Julian
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,US,Oregon,Paul Gregutt,Pinot Noir,65.0,87,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,Willamette Valley,Willamette Valley,@paulgwine,Sweet Cheeks


In [6]:
categorical_cols = ['title','country','province','taster_name','variety','description','designation','region_1','region_2','taster_twitter_handle','winery']
for column in categorical_cols:
    data1[column] = pandas.factorize(data1[column])[0]
data1.head(5)

,title,country,province,taster_name,variety,price,points,description,designation,region_1,region_2,taster_twitter_handle,winery
0,0,0,0,0,0,NaN,87,0,0,0,-1,0,0
1,1,1,1,1,1,15.0,87,1,1,-1,-1,1,1
2,2,2,2,2,2,14.0,87,2,-1,1,0,2,2
3,3,2,3,3,3,13.0,87,3,2,2,-1,-1,3
4,4,2,2,2,4,65.0,87,4,3,1,0,2,4


In [7]:
TempData = data1.drop('title',axis = 1)
TempData.head(5)

,country,province,taster_name,variety,price,points,description,designation,region_1,region_2,taster_twitter_handle,winery
0,0,0,0,0,NaN,87,0,0,0,-1,0,0
1,1,1,1,1,15.0,87,1,1,-1,-1,1,1
2,2,2,2,2,14.0,87,2,-1,1,0,2,2
3,2,3,3,3,13.0,87,3,2,2,-1,-1,3
4,2,2,2,4,65.0,87,4,3,1,0,2,4


In [9]:
print(TempData.corr(method="kendall"))

                        country  province  taster_name   variety     price  \
country                1.000000  0.147619     0.346385 -0.058916 -0.113747   
province               0.147619  1.000000     0.026458  0.178760 -0.097482   
taster_name            0.346385  0.026458     1.000000 -0.050132 -0.035870   
variety               -0.058916  0.178760    -0.050132  1.000000 -0.072780   
price                 -0.113747 -0.097482    -0.035870 -0.072780  1.000000   
points                -0.019066 -0.027057     0.072210 -0.056413  0.453928   
description            0.002969 -0.002915     0.001067 -0.001258  0.004391   
designation           -0.005016  0.050949     0.017445  0.005664  0.117950   
region_1              -0.131196  0.050349    -0.121019  0.123690  0.052187   
region_2              -0.218622 -0.331905     0.103569 -0.131286  0.086264   
taster_twitter_handle  0.328379  0.005706     0.909122 -0.015511 -0.029565   
winery                 0.029139  0.067450    -0.019315  0.071214

In [10]:
tempData = TempData.drop('description',axis = 1)
DataA = tempData.drop("region_1",axis=1)
DataB = DataA.drop("region_2",axis =1)
Datac= DataB.drop('taster_name',axis=1)
Datac['Target'] = Datac['points'].apply(lambda x: 1 if x > 88.0 else 0)

In [11]:
Datac.head(5)

,country,province,variety,price,points,designation,taster_twitter_handle,winery,Target
0,0,0,0,NaN,87,0,0,0,0
1,1,1,1,15.0,87,1,1,1,0
2,2,2,2,14.0,87,-1,2,2,0
3,2,3,3,13.0,87,2,-1,3,0
4,2,2,4,65.0,87,3,2,4,0


In [12]:
DataC = Datac.drop('points',axis=1)
DataC.head(5)

,country,province,variety,price,designation,taster_twitter_handle,winery,Target
0,0,0,0,NaN,0,0,0,0
1,1,1,1,15.0,1,1,1,0
2,2,2,2,14.0,-1,2,2,0
3,2,3,3,13.0,2,-1,3,0
4,2,2,4,65.0,3,2,4,0


In [13]:
Datad = DataC[['country','province','variety','price','designation','taster_twitter_handle','winery',"Target"]]
Datad.head(5)

,country,province,variety,price,designation,taster_twitter_handle,winery,Target
0,0,0,0,NaN,0,0,0,0
1,1,1,1,15.0,1,1,1,0
2,2,2,2,14.0,-1,2,2,0
3,2,3,3,13.0,2,-1,3,0
4,2,2,4,65.0,3,2,4,0


In [16]:
DataD = Datad[np.isfinite(Datad['price'])]
DataD.head(5)

,country,province,variety,price,designation,taster_twitter_handle,winery,Target
1,1,1,1,15.0,1,1,1,0
2,2,2,2,14.0,-1,2,2,0
3,2,3,3,13.0,2,-1,3,0
4,2,2,4,65.0,3,2,4,0
5,3,4,5,15.0,4,3,5,0


In [17]:
y = DataD["Target"].values

In [18]:
DataD.drop('Target', axis =1)

,country,province,variety,price,designation,taster_twitter_handle,winery
1,1,1,1,15.0,1,1,1
2,2,2,2,14.0,-1,2,2
3,2,3,3,13.0,2,-1,3
4,2,2,4,65.0,3,2,4
5,3,4,5,15.0,4,3,5
6,0,0,6,16.0,5,0,6
7,4,5,7,24.0,-1,1,7
8,5,6,7,12.0,6,-1,8
9,4,5,2,27.0,7,1,9
10,2,7,8,19.0,8,4,10


In [19]:
X = DataD.values

In [20]:
rfc = RandomForestClassifier(n_estimators=10,max_depth=None,random_state=None)

dtc = DecisionTreeClassifier(random_state=None)

svc = svm.SVC(random_state=None)

knn = KNeighborsClassifier(n_neighbors=10)

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=None)

In [21]:
accuracy_rfc = []
accuracy_dtc = []
accuracy_svc = []
accuracy_knn = []
accuracy_gbc = []

precision_rfc = []
precision_dtc = []
precision_svc = []
precision_knn = []
precision_gbc = []

recall_rfc = []
recall_dtc = []
recall_svc = []
recall_knn = []
recall_gbc = []

F1_rfc = []
F1_dtc = []
F1_svc = []
F1_knn = []
F1_gbc = []

In [24]:
scaler = preprocessing.StandardScaler().fit(X.astype(float))
X = scaler.transform(X.astype(float))

In [25]:
kf = KFold(n_splits=5,random_state = 0,shuffle = True)

In [26]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    rfc.fit(X_train, y_train)
    dtc.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    knn.fit(X_train, y_train)
    gbc.fit(X_train, y_train)
    
    y_pred_rfc = rfc.predict(X_test)
    y_pred_dtc = dtc.predict(X_test)
    y_pred_svc = svc.predict(X_test)
    y_pred_knn = knn.predict(X_test)
    
    y_pred_gbc = gbc.predict(X_test)
        
    accuracy_rfc.append( metrics.accuracy_score(y_test, y_pred_rfc))
    accuracy_dtc.append( metrics.accuracy_score(y_test, y_pred_dtc))
    accuracy_svc.append( metrics.accuracy_score(y_test, y_pred_svc))
    accuracy_knn.append( metrics.accuracy_score(y_test, y_pred_knn))
    accuracy_gbc.append( metrics.accuracy_score(y_test, y_pred_gbc))
    
    precision_rfc.append(metrics.precision_score(y_test, y_pred_rfc,average=None))
    precision_dtc.append(metrics.precision_score(y_test, y_pred_dtc,average=None))
    precision_svc.append(metrics.precision_score(y_test, y_pred_svc,average=None))
    precision_knn.append(metrics.precision_score(y_test, y_pred_knn,average=None))
    precision_gbc.append(metrics.precision_score(y_test, y_pred_gbc,average=None))
    
    recall_rfc.append(metrics.recall_score(y_test, y_pred_rfc,average=None))
    recall_dtc.append(metrics.recall_score(y_test, y_pred_dtc,average=None))
    recall_svc.append(metrics.recall_score(y_test, y_pred_svc,average=None))
    recall_knn.append(metrics.recall_score(y_test, y_pred_knn,average=None))
    recall_gbc.append(metrics.recall_score(y_test, y_pred_gbc,average=None))
    
    
    F1_rfc.append(metrics.f1_score(y_test, y_pred_rfc,average=None))
    F1_dtc.append(metrics.f1_score(y_test, y_pred_dtc,average=None))
    F1_svc.append(metrics.f1_score(y_test, y_pred_svc,average=None))
    F1_knn.append(metrics.f1_score(y_test, y_pred_knn,average=None))
    F1_gbc.append(metrics.f1_score(y_test, y_pred_gbc,average=None))

In [27]:
print ("  FOR Random Forest Classifier:")

print ("    Avg Accuracy : ",np.mean(accuracy_rfc))
print ("    Avg Precision : ",np.mean(precision_rfc))
print ("    Avg Recall : ",np.mean(recall_rfc))
print ("    Avg F1 : ",np.mean(F1_rfc))
print ("         ")
print ("         ")


print ("  FOR Decision Tree Classifier:")

print ("    Avg Accuracy : ",np.mean(accuracy_dtc))
print ("    Avg Precision : ",np.mean(precision_dtc))
print ("    Avg Recall : ",np.mean(recall_dtc))
print ("    Avg F1 : ",np.mean(F1_dtc))
print ("         ")
print ("         ")


print ("  FOR Support Vector Classification:")

print ("    Avg Accuracy : ",np.mean(accuracy_svc))
print ("    Avg Precision : ",np.mean(precision_svc))
print ("    Avg Recall : ",np.mean(recall_svc))
print ("    Avg F1 : ",np.mean(F1_svc))
print ("         ")
print ("         ")


print ("  FOR KNeighbors Classifier:")

print ("    Avg Accuracy : ",np.mean(accuracy_knn))
print ("    Avg Precision : ",np.mean(precision_knn))
print ("    Avg Recall : ",np.mean(recall_knn))
print ("    Avg F1 : ",np.mean(F1_knn))
print ("         ")
print ("         ")

print ("  FOR Gradient Boosting Classifier:")

print ("    Avg Accuracy : ",np.mean(accuracy_gbc))
print ("    Avg Precision : ",np.mean(precision_gbc))
print ("    Avg Recall : ",np.mean(recall_gbc))
print ("    Avg F1 : ",np.mean(F1_gbc))
print ("         ")
print ("         ")

  FOR Random Forest Classifier:
    Avg Accuracy :  1.0
    Avg Precision :  1.0
    Avg Recall :  1.0
    Avg F1 :  1.0
         
         
  FOR Decision Tree Classifier:
    Avg Accuracy :  1.0
    Avg Precision :  1.0
    Avg Recall :  1.0
    Avg F1 :  1.0
         
         
  FOR Support Vector Classification:
    Avg Accuracy :  0.999942136805125
    Avg Precision :  0.999939225135336
    Avg Recall :  0.999944593938882
    Avg F1 :  0.9999419043512182
         
         
  FOR KNeighbors Classifier:
    Avg Accuracy :  0.9994379003926431
    Avg Precision :  0.9994543291040102
    Avg Recall :  0.9994171684075885
    Avg F1 :  0.999435649970359
         
         
  FOR Gradient Boosting Classifier:
    Avg Accuracy :  1.0
    Avg Precision :  1.0
    Avg Recall :  1.0
    Avg F1 :  1.0
         
         


In [28]:
data.country.describe()

count     129908
unique        43
top           US
freq       54504
Name: country, dtype: object

In [29]:
data.price.describe()

count    120975.000000
mean         35.363389
std          41.022218
min           4.000000
25%          17.000000
50%          25.000000
75%          42.000000
max        3300.000000
Name: price, dtype: float64

In [30]:
data.points.describe()

count    129971.000000
mean         88.447138
std           3.039730
min          80.000000
25%          86.000000
50%          88.000000
75%          91.000000
max         100.000000
Name: points, dtype: float64

In [31]:
data.taster_name.describe()

count         103727
unique            19
top       Roger Voss
freq           25514
Name: taster_name, dtype: object

In [32]:
data.variety.describe()

count         129970
unique           707
top       Pinot Noir
freq           13272
Name: variety, dtype: object

In [33]:
data.province.describe()

count         129908
unique           425
top       California
freq           36247
Name: province, dtype: object